# 크롤러

- 1. 네이버
- 2. 구글
- 3. 게티
- 4. istock
- 5. 다음
- 6. 인스타

In [100]:
flist = ['비트수프']

# 네이버

In [ ]:
from selenium import webdriver
import requests
import urllib
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import re
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.webdriver.common.keys import Keys
import os

search_words = {"음식": flist}

value = 0
for name in search_words['음식']:
    driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
    driver.get('https://search.naver.com/search.naver?where=image&amp;sm=stb_nmr&amp;')
    elem = driver.find_element_by_id('nx_query')   #검색창 찾기
    
    search_name = name
    elem.send_keys(search_name)   #검색창에 찾을 이름 입력하기
    elem.submit()
    for i in range(1,10):
        driver.find_element_by_xpath("//body").send_keys(Keys.END)
        time.sleep(5)
    time.sleep(5)   #5초간 프로세스 정지, 시간 제공
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')     
    
    def fetch_list_url():
        params=[]
        imglist = soup.find_all('img',class_="_img")   #img에 해당하는 부분 찾기
        for im in imglist:
            params.append(im['src'])   #찾은 이미지들 params에 추가
        return params
    def fetch_detail_url():
        dir_name = "./img/" + search_name + '/'   #저정할 위치와 이름 지정
        if not os.path.isdir(dir_name):
            os.makedirs(dir_name)
        params = fetch_list_url()    
        a = 1  
        for p in params:
            urllib.request.urlretrieve(p, dir_name + search_name + '1_'+ str(a) + '.jpg')   #params에 추가된 모든 image들 저장
            a = a + 1    #저장할 이름의 숫자 키워가면서 
    fetch_detail_url()
driver.close()   #드라이브 종료

# 구글

In [101]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')


url = 'https://www.google.com/search?q='
detail = '&rlz=1C1NDCM_koKR830KR830&sxsrf=ALeKk00o2ZJazOr-YXIu7j7mXvKPAXycUg:1596970790605&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjG88Ll-43rAhXTMd4KHfzIDS4Q_AUoAXoECAwQAw&biw=1920&bih=903'


for search_list in flist:
    driver.get(url + search_list + detail)    #음식 리스트에 해당하는 이미지 페이지로 이동
    num_of_pagedowns = 60                     
    time.sleep(0.5)                    #시간 sleep 걸어주기 0.5초간
    body = driver.find_element_by_tag_name('body')        #body tag 찾기
    
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)     #페이지 스크롤 내리기
        time.sleep(0.5)
        num_of_pagedowns -= 1        
        
    for j,_ in enumerate(driver.find_elements_by_css_selector('img[class$=Q4LuWd]')[:500]):    #img들의 class에 해당하는 것 찾아서
        if _.get_attribute('src') !='':
            urllib.request.urlretrieve(_.get_attribute('src'),"./img/"+search_list+"/" + search_list +'2_'+str(j)+'.jpg') #지정 경로에 저장
                
    time.sleep(2)

driver.close()

# 게티

In [102]:
from selenium import webdriver
import urllib
import time
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
num=1
for search_list in flist:
    cumsum = 0
    driver.get('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q='+search_list+'&ssi=go#page='+str(num))  #음식 이미지에 해당하는 페이지로 이동
    while num<10:
        num+=1
        time.sleep(2)
        driver.find_element_by_xpath("//body").send_keys(Keys.END)
        if not driver.find_elements_by_css_selector('img[class=imgThumb]'):
            continue
        k = len(driver.find_elements_by_css_selector('img[class=imgThumb]'))   #image들의 공통된 class찾기
        for j, _ in enumerate(driver.find_elements_by_css_selector('img[class=imgThumb]')):   
            urllib.request.urlretrieve(_.get_attribute('src'),"./img/"+search_list+"/" + search_list +'3_'+ str(j+cumsum)+ '.jpg')  #저장경로에 저장
        cumsum+=k
        driver.get('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q='+search_list+'&ssi=go#page'+str(num))
driver.close()

# istock

In [104]:
from selenium import webdriver
import urllib
import time
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
for search_list in flist:
    num=1
    cumsum = 0
    driver.get('https://www.istockphoto.com/kr/이미지/'+search_list+'?mediatype=photography&phrase='+search_list+ '&sort=mostpopular&page='+str(num))
    while num<10:
        body = driver.find_element_by_tag_name('body')
        body.send_keys(Keys.END)
        time.sleep(2)
        num+=1
        k = len(driver.find_elements_by_xpath('//figure[@ng-class="figureClassNames"]//img'))   #xpath로 image에 해당하는 것 찾기
        for j, _ in enumerate(driver.find_elements_by_xpath('//figure[@ng-class="figureClassNames"]//img')):
            if _.get_attribute('src'):
                urllib.request.urlretrieve(_.get_attribute('src'),"./img/" + search_list + '/' + search_list +'4_'+ str(j+cumsum)+ '.jpg') #사진저장
        cumsum+=k
        driver.get('https://www.istockphoto.com/kr/이미지/'+search_list+'?mediatype=photography&phrase='+search_list+ '&sort=mostpopular&page='+str(num))

driver.close()

# 다음

In [105]:
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')


url = 'https://search.daum.net/search?w=img&nil_search=btn&DA=NTB&enc=utf8&q='
num=2
down = 8
for search_list in flist:
    driver.get(url + search_list)
    time.sleep(1)
    while num:
        num-=1
        body = driver.find_element_by_tag_name('body')
        time.sleep(2)
        while down:
            body.send_keys(Keys.END)
            time.sleep(0.5)
            down-=1
            if down%4==0:    
                driver.find_element_by_xpath('//div[@class="extend_comp extend_imgtab"]//a').click()  #더보기의 의미인 화살표 누르기
    for j,_ in enumerate(driver.find_elements_by_css_selector('img[class=thumb_img]')[:500]):   #image class가 thumb_img 모두 찾아서
        if _.get_attribute('src') !='':
            urllib.request.urlretrieve(_.get_attribute('src'),"./img/"+search_list+"/" + search_list +'5_'+str(j)+'.jpg')  #저장
                
    time.sleep(2)

driver.close()